In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import DistilBertTokenizer, TFDistilBertModel
from tensorflow.keras.layers import Input, Dense, LSTM, Bidirectional
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

# Load the dataset
train_data = pd.read_csv('final_train.csv')
test_data = pd.read_csv('final_test.csv')

# Tokenize text data using DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
max_len = 100  # Maximum sequence length
X_train_text = tokenizer(train_data['text'].tolist(), padding='max_length', truncation=True, max_length=max_len, return_tensors='tf')
X_test_text = tokenizer(test_data['text'].tolist(), padding='max_length', truncation=True, max_length=max_len, return_tensors='tf')

# Convert labels to numerical format
label_mapping = {'bully-Spam': 0, 'not_bully-Spam': 1, 'bully-Ham': 2, 'not_bully-Ham': 3}
y_train = train_data['new_label'].map(label_mapping)
y_test = test_data['new_label'].map(label_mapping)

# Define DistilBERT model
def build_distil_bert_model():
    input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
    attention_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")

    distil_bert_model = TFDistilBertModel.from_pretrained("distilbert-base-uncased")
    distil_bert_outputs = distil_bert_model(input_ids, attention_mask=attention_mask)[0]

    return Model(inputs=[input_ids, attention_mask], outputs=distil_bert_outputs)

# Build DistilBERT model
distil_bert_model = build_distil_bert_model()

# Combine DistilBERT and Bi-LSTM models
def build_combined_model(distil_bert_model):
    input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
    attention_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")

    distil_bert_output = distil_bert_model([input_ids, attention_mask])
    
    # Add Bi-LSTM layer
    bi_lstm_output = Bidirectional(LSTM(128))(distil_bert_output)
    
    output = Dense(4, activation='softmax')(bi_lstm_output)

    return Model(inputs=[input_ids, attention_mask], outputs=output)

# Build combined model
combined_model = build_combined_model(distil_bert_model)

# Compile combined model with a smaller learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)  # Adjust the learning rate as needed
combined_model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train combined model with a smaller batch size
history = combined_model.fit(
    [X_train_text['input_ids'], X_train_text['attention_mask']], 
    y_train, 
    epochs=5, 
    batch_size=16, 
    validation_split=0.1
)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Epoch 1/5
79/79 [==============================] - 200s 2s/step - loss: 0.7686 - accuracy: 0.6868 - val_loss: 0.6311 - val_accuracy: 0.7376
Epoch 2/5
79/79 [==============================] - 187s 2s/step - loss: 0.3718 - accuracy: 0.8699 - val_loss: 0.3119 - val_accuracy: 0.8652
Epoch 3/5
79/79 [==============================] - 188s 2s/step - loss: 0.2185 - accuracy: 0.9278 - val_loss: 0.1734 - val_accuracy: 0.9433
Epoch 4/5
79/79 [==============================] - 190s 2s/step - loss: 0.1262 - accuracy: 0.9651 - val_loss: 0.1175 - val_accuracy: 0.9504
Epoch 5/5
79/79 [==============================] - 199s 3s/step - loss: 0.0788 - accuracy: 0.9738 - val_loss: 0.1803 - val_accuracy: 0.9220


In [3]:
# Make predictions on test data
predictions = combined_model.predict([X_test_text['input_ids'], X_test_text['attention_mask']])

# Convert predicted probabilities to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Evaluate the model
from sklearn.metrics import classification_report

# Convert test labels to numpy array
y_test_np = np.array(y_test)

# Print classification report
print(classification_report(y_test_np, predicted_labels))

14/14 [==============================] - 18s 1s/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       0.95      0.91      0.93       115
           2       0.89      0.83      0.86       103
           3       0.87      0.91      0.89       198

    accuracy                           0.90       426
   macro avg       0.93      0.92      0.92       426
weighted avg       0.90      0.90      0.90       426

